In [1]:
from pathlib import Path
import os
import os.path as osp
import sys
import math
import gc
import pickle
from collections import defaultdict
import json
import enum

In [2]:
import pandas as pd

In [3]:
from tqdm.notebook import tqdm

In [248]:
from sqlalchemy import create_engine, Column, Table, MetaData, ForeignKey
from sqlalchemy import Integer, String, Date, DateTime, Float, Boolean, Text, Enum
from sqlalchemy.orm import sessionmaker, declarative_base, mapped_column

In [249]:
csv_root = Path("../Holdout_final")
csv_files = list(p for p in csv_root.iterdir() if p.suffix == ".csv")
print(f"{len(csv_files)} found:")
for fp in csv_files:
    print(f"- {fp}")

14 found:
- ../Holdout_final/Pharmacy Utilization_Holdout.csv
- ../Holdout_final/Social Determinants of Health_Holdout.csv
- ../Holdout_final/QUALITY_DATA_Holdout.csv
- ../Holdout_final/Demographics_Holdout.csv
- ../Holdout_final/Sales Channel_Holdout.csv
- ../Holdout_final/Control Point_Holdout.csv
- ../Holdout_final/Additional Features_Holdout.csv
- ../Holdout_final/Web Activity_Holdout.csv
- ../Holdout_final/humana_mays_target_members_Holdout.csv
- ../Holdout_final/humana_mays_target_member_conditions_Holdout.csv
- ../Holdout_final/humana_mays_target_member_visit_claims_Holdout.csv
- ../Holdout_final/Cost & Utilization_Holdout.csv
- ../Holdout_final/humana_mays_target_member_details_Holdout.csv
- ../Holdout_final/MEMBER_DATA_Holdout.csv


In [256]:
dataframes = {}

def add_dataframes(csv_files, datas_type):
    print("Adding dataframes (type: '{}')".format(datas_type))
    for icsv, csv_path in enumerate(csv_files):
        name = csv_path.stem
        if name.endswith("_Holdout"): name = name[:-len("_Holdout")]
        print("[{}/{}] Reading csv for '{}' ({})".format(icsv+1, len(csv_files), name, csv_path.stem))
        df = pd.read_csv(csv_path)
        df['type'] = datas_type
        if name in dataframes:
            dataframes[name] = pd.concat([dataframes[name], df], axis=0)
        else:
            dataframes[name] = df
        print("  DF shape: {}".format(dataframes[name].shape))
        gc.collect()
    print()

csv_root = Path("../Training_final")
csv_files = list(p for p in csv_root.iterdir() if p.suffix == ".csv")
add_dataframes(csv_files, "Training")

csv_root = Path("../Holdout_final")
csv_files = list(p for p in csv_root.iterdir() if p.suffix == ".csv")
add_dataframes(csv_files, "Holdout")

Adding dataframes (type: 'Training')
[1/14] Reading csv for 'humana_mays_target_members' (humana_mays_target_members)
  DF shape: (1527904, 6)
[2/14] Reading csv for 'Control Point' (Control Point)
  DF shape: (1527904, 67)
[3/14] Reading csv for 'Cost & Utilization' (Cost & Utilization)
  DF shape: (1527904, 37)
[4/14] Reading csv for 'Additional Features' (Additional Features)
  DF shape: (1527904, 13)
[5/14] Reading csv for 'humana_mays_target_member_conditions' (humana_mays_target_member_conditions)
  DF shape: (4009342, 8)
[6/14] Reading csv for 'humana_mays_target_member_details' (humana_mays_target_member_details)
  DF shape: (1527904, 14)
[7/14] Reading csv for 'Pharmacy Utilization' (Pharmacy Utilization)
  DF shape: (1527904, 18)
[8/14] Reading csv for 'Sales Channel' (Sales Channel)
  DF shape: (1527904, 3)
[9/14] Reading csv for 'humana_mays_target_member_visit_claims' (humana_mays_target_member_visit_claims)


/tmp/ipykernel_9863/3785103976.py:9: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


  DF shape: (19456796, 28)
[10/14] Reading csv for 'Demographics' (Demographics)
  DF shape: (1527904, 8)
[11/14] Reading csv for 'Social Determinants of Health' (Social Determinants of Health)
  DF shape: (1527904, 79)
[12/14] Reading csv for 'Web Activity' (Web Activity)
  DF shape: (1527904, 16)
[13/14] Reading csv for 'QUALITY_DATA' (QUALITY_DATA)


/tmp/ipykernel_9863/3785103976.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


  DF shape: (33572241, 9)
[14/14] Reading csv for 'MEMBER_DATA' (MEMBER_DATA)
  DF shape: (1527904, 8)

Adding dataframes (type: 'Holdout')
[1/14] Reading csv for 'Pharmacy Utilization' (Pharmacy Utilization_Holdout)
  DF shape: (1909880, 18)
[2/14] Reading csv for 'Social Determinants of Health' (Social Determinants of Health_Holdout)
  DF shape: (1909880, 79)
[3/14] Reading csv for 'QUALITY_DATA' (QUALITY_DATA_Holdout)


/tmp/ipykernel_9863/3785103976.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


  DF shape: (41950227, 9)
[4/14] Reading csv for 'Demographics' (Demographics_Holdout)
  DF shape: (1909880, 8)
[5/14] Reading csv for 'Sales Channel' (Sales Channel_Holdout)
  DF shape: (1909880, 3)
[6/14] Reading csv for 'Control Point' (Control Point_Holdout)
  DF shape: (1909880, 67)
[7/14] Reading csv for 'Additional Features' (Additional Features_Holdout)
  DF shape: (1909880, 13)
[8/14] Reading csv for 'Web Activity' (Web Activity_Holdout)
  DF shape: (1909880, 16)
[9/14] Reading csv for 'humana_mays_target_members' (humana_mays_target_members_Holdout)
  DF shape: (1909880, 6)
[10/14] Reading csv for 'humana_mays_target_member_conditions' (humana_mays_target_member_conditions_Holdout)
  DF shape: (5011114, 8)
[11/14] Reading csv for 'humana_mays_target_member_visit_claims' (humana_mays_target_member_visit_claims_Holdout)


/tmp/ipykernel_9863/3785103976.py:9: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


  DF shape: (24314072, 28)
[12/14] Reading csv for 'Cost & Utilization' (Cost & Utilization_Holdout)
  DF shape: (1909880, 37)
[13/14] Reading csv for 'humana_mays_target_member_details' (humana_mays_target_member_details_Holdout)
  DF shape: (1909880, 14)
[14/14] Reading csv for 'MEMBER_DATA' (MEMBER_DATA_Holdout)
  DF shape: (1909880, 8)



In [250]:
ids_training = set(dataframes['humana_mays_target_members'][dataframes['humana_mays_target_members']['type'] == 'Training']['id'])
ids_holdout = set(dataframes['humana_mays_target_members'][dataframes['humana_mays_target_members']['type'] == 'Holdout']['id'])
assert len(ids_training.intersection(ids_holdout)) == 0

print("Verifying Training/Holdout status...")
for name, df in tqdm(dataframes.items()):
    ids_training_cur = set(df[df['type'] == 'Training']['id'])
    ids_holdout_cur = set(df[df['type'] == 'Holdout']['id'])
    assert len(ids_training_cur) < len(ids_training) or ids_training_cur == ids_training
    assert len(ids_holdout_cur) < len(ids_holdout) or ids_holdout_cur == ids_holdout

Verifying Training/Holdout status...


  0%|          | 0/14 [00:00<?, ?it/s]

1152178

In [54]:
len(ids_training)

1527904

In [11]:
print("Dataframes (#={}): {}".format(len(dataframes), list(dataframes.keys())))

Dataframes (#=14): ['humana_mays_target_members', 'Control Point', 'Cost & Utilization', 'Additional Features', 'humana_mays_target_member_conditions', 'humana_mays_target_member_details', 'Pharmacy Utilization', 'Sales Channel', 'humana_mays_target_member_visit_claims', 'Demographics', 'Social Determinants of Health', 'Web Activity', 'QUALITY_DATA', 'MEMBER_DATA']


# Analyse CSVs, insert into db

## Database

In [251]:
Base = declarative_base()
conn_str = "sqlite:///out/db/db1.sqlite"
db_engine = create_engine(conn_str)
Base.metadata.create_all(db_engine)
print("Initialized database engine")

Initialized database engine


In [252]:
Session = sessionmaker(bind=db_engine)

In [253]:
class EType(enum.Enum):
    Training = 'Training'
    Holdout = 'Holdout'

def get_etype(s):
    return {'Training': EType.Training, 'Holdout': EType.Holdout}[s]

In [254]:
class Member(Base):
    __tablename__ = 'member'

    member_id = mapped_column(Integer, primary_key=True)
    data_type = Column(Enum(EType))

In [65]:
Base.metadata.create_all(db_engine)

In [ ]:
with Session() as sess:
    for id_training in tqdm(ids_training):
        e = Member()
        e.member_id = id_training
        e.data_type = EType.Training
        sess.add(e)
        sess.commit()

    for id_training in tqdm(ids_training):
        e = Member()
        e.member_id = id_training
        e.data_type = EType.Training
        sess.add(e)
        sess.commit()

## Target members

In [15]:
df = dataframes['humana_mays_target_members']
display(df[df['type'] == "Holdout"])

,calendar_year,product_type,plan_category,preventive_visit_gap_ind,id,type
0,2023,LPPO,Medicare Advantage,NaN,1387512,Holdout
1,2023,LPPO,Medicare Advantage,NaN,787578,Holdout
2,2023,LPPO,Medicare Advantage,NaN,1362865,Holdout
3,2023,LPPO,Medicare Advantage,NaN,1400934,Holdout
4,2023,LPPO,Medicare Advantage,NaN,274872,Holdout
...,...,...,...,...,...,...
381971,2023,LPPO,Medicare Advantage,NaN,1691098,Holdout
381972,2023,LPPO,Medicare Advantage,NaN,63411,Holdout
381973,2023,LPPO,Medicare Advantage,NaN,1464366,Holdout
381974,2023,LPPO,Medicare Advantage,NaN,715653,Holdout


In [16]:
print("Calender year type: {}, min: {}, max: {}".format(df['calendar_year'].dtype, df['calendar_year'].min(), df['calendar_year'].max()))

Calender year type: int64, min: 2023, max: 2023


In [17]:
print(f"{df['product_type'].dtype=}")
print(f"{df['product_type'].unique()=}")

df['product_type'].dtype=dtype('O')
df['product_type'].unique()=array(['LPPO'], dtype=object)


In [18]:
print(f"{df['plan_category'].dtype=}")
print(f"{df['plan_category'].unique()=}")

df['plan_category'].dtype=dtype('O')
df['plan_category'].unique()=array(['Medicare Advantage'], dtype=object)


In [19]:
print(f"{df['preventive_visit_gap_ind']=}")
print(f"{df['preventive_visit_gap_ind'].unique()=}")

df['preventive_visit_gap_ind']=0         0.0
1         0.0
2         1.0
3         0.0
4         0.0
         ... 
381971    NaN
381972    NaN
381973    NaN
381974    NaN
381975    NaN
Name: preventive_visit_gap_ind, Length: 1909880, dtype: float64
df['preventive_visit_gap_ind'].unique()=array([ 0.,  1., nan])


In [20]:
df['id']

0          380667
1          378860
2         1370287
3          184540
4         1628837
           ...   
381971    1691098
381972      63411
381973    1464366
381974     715653
381975     940304
Name: id, Length: 1909880, dtype: int64

In [21]:
df['type']

0         Training
1         Training
2         Training
3         Training
4         Training
            ...   
381971     Holdout
381972     Holdout
381973     Holdout
381974     Holdout
381975     Holdout
Name: type, Length: 1909880, dtype: object

In [222]:
class EProductType(enum.Enum):
    LPPO = 'LPPO'

class EPlanCategory(enum.Enum):
    MedicareAdvantage = 'Medicate Advantage'

class RawTargetMembers(Base):
    __tablename__ = 'raw_target_members'

    id = Column(Integer, autoincrement=True, primary_key=True)
    member_id = mapped_column(Integer, ForeignKey("member.member_id"))
    calendar_year = Column(Integer)
    product_type = Column(Enum(EProductType))
    plan_category = Column(Enum(EPlanCategory))
    preventive_visit_gap_ind = Column(Boolean)

In [230]:
for var in dir(RawTargetMembers):
    print(var)
    if isinstance(var, Integer):
        print(var)

__abstract__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__mapper__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__table__
__tablename__
__weakref__
_sa_class_manager
_sa_registry
calendar_year
id
member_id
metadata
plan_category
product_type
registry


In [237]:
isinstance(RawTargetMembers.id.type, Integer)

True

In [244]:
dir(RawTargetMembers.product_type.type.enum_class)

['LPPO', '__class__', '__doc__', '__members__', '__module__']

In [242]:
dir(RawTargetMembers.product_type.type)

['Comparator',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__visit_name__',
 '__weakref__',
 '_cached_bind_processor',
 '_cached_custom_processor',
 '_cached_literal_processor',
 '_cached_result_processor',
 '_compare_type_affinity',
 '_compiler_dispatch',
 '_create_events',
 '_db_value_for_elem',
 '_default_dialect',
 '_default_length',
 '_dialect_info',
 '_enum_init',
 '_enums_argument',
 '_gen_dialect_impl',
 '_generate_compiler_dispatch',
 '_generic_type_affinity',
 '_has_bind_expression',
 '_has_column_expression',
 '_is_array',
 '_is_impl_for_variant',
 '_is_protocol',
 '_is_table_

In [45]:
EProductType.__members__

mappingproxy({'LPPO': <EProductType.LPPO: 'LPPO'>})

In [46]:
Base.metadata.create_all(db_engine)

In [ ]:
with Session() as sess:
    for i in tqdm(range(len(df))):
        d = df.iloc[i]
        e = RawTargetMembers()
        e.member_id = int(d['id'])
        e.calendar_year = int(d['calendar_year'])
        e.product_type = {'LPPO': EProductType.LPPO}[d['product_type']]
        e.plan_category = {'Medicare Advantage': EPlanCategory.MedicareAdvantage}[d['plan_category']]
        try:
            sess.add(e)
            sess.commit()
        except:
            sess.rollback()
            raise

## Additional Features

In [270]:
df = dataframes['Additional Features']

cci_score
dcsi_score
fci_score
cms_tot_partd_payment_amt
cms_tot_ma_payment_amt
cms_frailty_ind
atlas_grocpth14
atlas_povertyallagespct
atlas_recfacpth14
atlas_ffrpth14
atlas_fsrpth14

In [258]:
df['cci_score']

0         7.0
1         4.0
2         3.0
3         4.0
4         4.0
         ... 
381971    3.0
381972    2.0
381973    1.0
381974    2.0
381975    2.0
Name: cci_score, Length: 1909880, dtype: float64

In [259]:
df['dcsi_score'] # should be int
print(f"{df['dcsi_score'].isna().sum()=}")

df['dcsi_score'].isna().sum()=np.int64(629)


In [260]:
df['dcsi_score'] # should be int
print(f"{df['fci_score'].isna().sum()=}")

df['fci_score'].isna().sum()=np.int64(629)


In [261]:
df['cms_tot_partd_payment_amt']

0            NaN
1         178.83
2         150.99
3         161.81
4          44.03
           ...  
381971       NaN
381972       NaN
381973       NaN
381974       NaN
381975       NaN
Name: cms_tot_partd_payment_amt, Length: 1909880, dtype: float64

In [266]:
pd.api.types.is_float_dtype(df['cms_tot_partd_payment_amt'].dtype)

True

In [269]:
pd.isna(float('inf'))

False

In [80]:
df['cms_tot_ma_payment_amt']

0             NaN
1         1103.23
2          398.96
3          967.76
4          849.85
           ...   
381971        NaN
381972        NaN
381973        NaN
381974        NaN
381975        NaN
Name: cms_tot_ma_payment_amt, Length: 1909880, dtype: float64

In [82]:
df['cms_frailty_ind'].unique() # should be int in {0, 1}

array([ 1.,  0., nan])

In [84]:
for colname in ['atlas_grocpth14', 'atlas_povertyallagespct', 'atlas_recfacpth14', 'atlas_ffrpth14', 'atlas_fsrpth14']:
    print(colname)
    print(df[colname].dtype)
    print()


atlas_grocpth14
float64

atlas_povertyallagespct
float64

atlas_recfacpth14
float64

atlas_ffrpth14
float64

atlas_fsrpth14
float64



In [85]:
df

,cms_frailty_ind,cms_tot_ma_payment_amt,cms_tot_partd_payment_amt,atlas_recfacpth14,atlas_ffrpth14,atlas_fsrpth14,atlas_grocpth14,atlas_povertyallagespct,cci_score,fci_score,dcsi_score,id,type
0,1.0,NaN,NaN,0.048228,0.569086,0.356884,0.183265,17.5,7.0,5.0,4.0,1352611,Training
1,0.0,1103.23,178.83,0.039119,0.508548,0.782381,0.117357,12.1,4.0,5.0,4.0,299881,Training
2,0.0,398.96,150.99,0.118075,0.688773,0.551018,0.098396,15.9,3.0,1.0,0.0,1191826,Training
3,0.0,967.76,161.81,0.040908,0.368173,0.204541,0.286357,22.9,4.0,4.0,0.0,1772108,Training
4,0.0,849.85,44.03,0.000000,0.682271,0.367377,0.472342,30.4,4.0,3.0,4.0,171691,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381971,1.0,NaN,NaN,0.136716,0.841012,0.880369,0.167788,10.9,3.0,3.0,0.0,914377,Holdout
381972,1.0,NaN,NaN,0.096105,0.701701,0.687219,0.213275,8.0,2.0,2.0,0.0,885778,Holdout
381973,1.0,NaN,NaN,0.108904,0.680650,0.535445,0.149743,9.8,1.0,0.0,0.0,374571,Holdout
381974,1.0,NaN,NaN,0.133044,0.933523,1.124218,0.168522,8.3,2.0,0.0,0.0,753477,Holdout


In [ ]:
class RawAdditionalFeaturesRaw(Base):
    __tablename__ = 'raw_additional_features'

    member_id = mapped_column(ForeignKey("member.member_id"))

    cci_score = Column(Float)
    dcsi_score = Column(Integer)
    fci_score = Column(Integer)
    cms_tot_partd_payment_amt = Column(Float)
    cms_tot_ma_payment_amt = Column(Float)
    cms_frailty_ind = Column(Boolean)
    atlas_grocpth14 = Column(Float)
    atlas_povertyallagespct = Column(Float)
    atlas_recfacpth14 = Column(Float)
    atlas_ffrpth14 = Column(Float)
    atlas_fsrpth14 = Column(Float)

## Marketing Control Point

In [89]:
df = dataframes['Control Point']
df

,cnt_cp_emails_0,cnt_cp_emails_1,cnt_cp_emails_2,cnt_cp_emails_3,cnt_cp_emails_4,cnt_cp_emails_5,cnt_cp_emails_6,cnt_cp_emails_7,cnt_cp_emails_8,cnt_cp_emails_9,...,cnt_cp_livecall_9,cnt_cp_livecall_10,cnt_cp_livecall_11,cnt_cp_emails_pmpm_ct,cnt_cp_print_pmpm_ct,cnt_cp_vat_pmpm_ct,cnt_cp_webstatement_pmpm_ct,cnt_cp_livecall_pmpm_ct,id,type
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.000000,0.000000,11.500000,0.0,1352611,Training
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.500000,1.000000,2.416667,0.0,299881,Training
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.166667,1.000000,2.166667,0.0,1191826,Training
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.083333,2.166667,1.583333,0.0,1772108,Training
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.083333,0.250000,4.250000,0.0,171691,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.000000,3.000000,0.0,914377,Holdout
381972,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,4.000000,0.000000,7.000000,0.0,885778,Holdout
381973,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,4.000000,1.000000,5.000000,0.0,374571,Holdout
381974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,753477,Holdout


In [ ]:
class RawMarketingControlPoint(Base):
    __tablename__ = 'raw_marketing_control_point'

    member_id = mapped_column(ForeignKey("member.member_id"))

    cnt_cp_emails_0 = Column(Integer)
    cnt_cp_emails_1 = Column(Integer)
    cnt_cp_emails_10 = Column(Integer)
    cnt_cp_emails_11 = Column(Integer)
    cnt_cp_emails_2 = Column(Integer)
    cnt_cp_emails_3 = Column(Integer)
    cnt_cp_emails_4 = Column(Integer)
    cnt_cp_emails_5 = Column(Integer)
    cnt_cp_emails_6 = Column(Integer)
    cnt_cp_emails_7 = Column(Integer)
    cnt_cp_emails_8 = Column(Integer)
    cnt_cp_emails_9 = Column(Integer)
    cnt_cp_emails_pmpm_ct = Column(Float)
    cnt_cp_livecall_0 = Column(Integer)
    cnt_cp_livecall_1 = Column(Integer)
    cnt_cp_livecall_10 = Column(Integer)
    cnt_cp_livecall_11 = Column(Integer)
    cnt_cp_livecall_2 = Column(Integer)
    cnt_cp_livecall_3 = Column(Integer)
    cnt_cp_livecall_4 = Column(Integer)
    cnt_cp_livecall_5 = Column(Integer)
    cnt_cp_livecall_6 = Column(Integer)
    cnt_cp_livecall_7 = Column(Integer)
    cnt_cp_livecall_8 = Column(Integer)
    cnt_cp_livecall_9 = Column(Integer)
    cnt_cp_livecall_pmpm_ct = Column(Float)
    cnt_cp_print_0 = Column(Integer)
    cnt_cp_print_1 = Column(Integer)
    cnt_cp_print_10 = Column(Integer)
    cnt_cp_print_11 = Column(Integer)
    cnt_cp_print_2 = Column(Integer)
    cnt_cp_print_3 = Column(Integer)
    cnt_cp_print_4 = Column(Integer)
    cnt_cp_print_5 = Column(Integer)
    cnt_cp_print_6 = Column(Integer)
    cnt_cp_print_7 = Column(Integer)
    cnt_cp_print_8 = Column(Integer)
    cnt_cp_print_9 = Column(Integer)
    cnt_cp_print_pmpm_ct = Column(Float)
    cnt_cp_vat_0 = Column(Integer)
    cnt_cp_vat_1 = Column(Integer)
    cnt_cp_vat_10 = Column(Integer)
    cnt_cp_vat_11 = Column(Integer)
    cnt_cp_vat_2 = Column(Integer)
    cnt_cp_vat_3 = Column(Integer)
    cnt_cp_vat_4 = Column(Integer)
    cnt_cp_vat_5 = Column(Integer)
    cnt_cp_vat_6 = Column(Integer)
    cnt_cp_vat_7 = Column(Integer)
    cnt_cp_vat_8 = Column(Integer)
    cnt_cp_vat_9 = Column(Integer)
    cnt_cp_vat_pmpm_ct = Column(Float)
    cnt_cp_webstatement_0 = Column(Integer)
    cnt_cp_webstatement_1 = Column(Integer)
    cnt_cp_webstatement_10 = Column(Integer)
    cnt_cp_webstatement_11 = Column(Integer)
    cnt_cp_webstatement_2 = Column(Integer)
    cnt_cp_webstatement_3 = Column(Integer)
    cnt_cp_webstatement_4 = Column(Integer)
    cnt_cp_webstatement_5 = Column(Integer)
    cnt_cp_webstatement_6 = Column(Integer)
    cnt_cp_webstatement_7 = Column(Integer)
    cnt_cp_webstatement_8 = Column(Integer)
    cnt_cp_webstatement_9 = Column(Integer)
    cnt_cp_webstatement_pmpm_ct = Column(Float)

## Cost & Utilization

In [95]:
df = dataframes['Cost & Utilization']
df

,nonpar_clm_ct_pmpm,nonpar_allowed_pmpm_cost,nonpar_net_paid_pmpm_cost,nonpar_cob_paid_pmpm_cost,nonpar_coins_pmpm_cost,nonpar_copay_pmpm_cost,nonpar_deduct_pmpm_cost,nonpar_mbr_resp_pmpm_cost,nonpar_ds_clm,oontwk_clm_ct_pmpm,...,bh_rtc_admit_days_pmpm,bh_psyc_visit_ct_pmpm,total_ip_acute_admit_days_pmpm,total_ip_ltach_admit_days_pmpm,total_ip_maternity_admit_days_pmpm,total_ip_mhsa_admit_days_pmpm,total_ip_rehab_admit_days_pmpm,total_ip_snf_admit_days_pmpm,id,type
0,1.000000,201.670000,197.635000,0.0,0.00,0.0,0.0,0.00,40.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1352611,Training
1,0.166667,9.913333,9.913333,0.0,0.00,0.0,0.0,0.00,101.0,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,299881,Training
2,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,480.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1191826,Training
3,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,480.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1772108,Training
4,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,480.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,171691,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381971,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,480.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,914377,Holdout
381972,1.000000,168.440000,107.300000,0.0,58.95,0.0,0.0,58.95,16.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,885778,Holdout
381973,6.000000,871.610000,643.940000,0.0,124.53,90.0,0.0,214.53,10.0,6.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,374571,Holdout
381974,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,480.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,753477,Holdout


In [ ]:
class RawCostAndUtilization(Base):
    __tablename__ = 'raw_cost_and_utilization'

    member_id = mapped_column(ForeignKey("member.member_id"))

    bh_psyc_visit_ct_pmpm = Column(Float)
    bh_rtc_admit_ct_pmpm = Column(Float)
    bh_rtc_admit_days_pmpm = Column(Float)
    days_since_last_clm = Column(Integer)
    nonpar_allowed_pmpm_cost = Column(Float)
    nonpar_clm_ct_pmpm = Column(Float)
    nonpar_cob_paid_pmpm_cost = Column(Float)
    nonpar_coins_pmpm_cost = Column(Float)
    nonpar_copay_pmpm_cost = Column(Float)
    nonpar_deduct_pmpm_cost = Column(Float)
    nonpar_ds_clm = Column(Integer)
    nonpar_mbr_resp_pmpm_cost = Column(Float)
    nonpar_net_paid_pmpm_cost = Column(Float)
    oontwk_allowed_pmpm_cost = Column(Float)
    oontwk_clm_ct_pmpm = Column(Float)
    oontwk_cob_paid_pmpm_cost = Column(Float)
    oontwk_coins_pmpm_cost = Column(Float)
    oontwk_copay_pmpm_cost = Column(Float)
    oontwk_deduct_pmpm_cost = Column(Float)
    oontwk_ds_clm = Column(Integer)
    oontwk_mbr_resp_pmpm_cost = Column(Float)
    oontwk_net_paid_pmpm_cost = Column(Float)
    total_allowed_pmpm_cost = Column(Float)
    total_cob_paid_pmpm_cost = Column(Float)
    total_coins_pmpm_cost = Column(Float)
    total_copay_pmpm_cost = Column(Float)
    total_deduct_pmpm_cost = Column(Float)
    total_ip_acute_admit_days_pmpm = Column(Float)
    total_ip_ltach_admit_days_pmpm = Column(Float)
    total_ip_maternity_admit_days_pmpm = Column(Float)
    total_ip_mhsa_admit_days_pmpm = Column(Float)
    total_ip_rehab_admit_days_pmpm = Column(Float)
    total_ip_snf_admit_days_pmpm = Column(Float)
    total_mbr_resp_pmpm_cost = Column(Float)
    total_net_paid_pmpm_cost = Column(Float)

## Demographics

In [98]:
df = dataframes['Demographics']
df

,lang_spoken_cd,rucc_category,riskarr_downside,riskarr_upside,riskarr_rewards,riskarr_global,id,type
0,ENG,1-Metro,0.0,1.0,0.0,0.0,1352611,Training
1,ENG,2-Metro,0.0,0.0,0.0,0.0,299881,Training
2,NaN,2-Metro,0.0,0.0,0.0,0.0,1191826,Training
3,ENG,7-Nonmetro,0.0,1.0,0.0,0.0,1772108,Training
4,NaN,7-Nonmetro,0.0,0.0,0.0,0.0,171691,Training
...,...,...,...,...,...,...,...,...
381971,ENG,2-Metro,1.0,0.0,0.0,1.0,914377,Holdout
381972,ENG,1-Metro,1.0,0.0,0.0,0.0,885778,Holdout
381973,ENG,2-Metro,0.0,0.0,0.0,0.0,374571,Holdout
381974,NaN,1-Metro,1.0,0.0,0.0,1.0,753477,Holdout


In [99]:
df['lang_spoken_cd'].unique()

array(['ENG', nan, 'OTH', 'SPA', 'CPF', 'CHI', 'KOR', 'FRE', 'VIE', 'YUE',
       'FAS', 'PER', 'JPN', 'POL', 'RUS', 'POR', 'TGL', 'ARA', 'ITA',
       'GER', 'CRE', 'CMN', 'LAO', 'DUT', 'THA', 'NAV', 'ZZZ', 'TAG',
       'HMG', 'BEN', 'PHI', 'MAN', 'HIN', 'GUJ', 'SRP', 'IRA', 'URD'],
      dtype=object)

In [100]:
df['rucc_category'].unique()

array(['1-Metro', '2-Metro', '7-Nonmetro', '9-Nonmetro', '6-Nonmetro',
       '3-Metro', '4-Nonmetro', '5-Nonmetro', '8-Nonmetro', nan],
      dtype=object)

In [101]:
df['riskarr_downside'].unique()

array([ 0.,  1., nan])

In [102]:
df['riskarr_upside'].unique()

array([ 1.,  0., nan])

In [103]:
df['riskarr_rewards'].unique()

array([ 0.,  1., nan])

In [104]:
df['riskarr_global'].unique()

array([ 0.,  1., nan])

In [271]:
class ERUCC(enum.Enum):
    Metro_1 = '1-Metro'
    Metro_2 = '2-Metro'
    Metro_3 = '3-Metro'
    Metro_4 = '4-Nonmetro'
    Nonmetro_5 = '5-Nonmetro'
    Nonmetro_6 = '6-Nonmetro'
    Nonmetro_7 = '7-Nonmetro'
    Nonmetro_8 = '8-Nonmetro'
    Nonemtro_9 = '9-Nonmetro'

class ELangSpoken(enum.Enum):
    ENG = 'ENG'; OTH = 'OTH'; SPA = 'SPA'; CPF = 'CPF'; CHI = 'CHI'; KOR = 'KOR'; FRE = 'FRE'
    VIE = 'VIE'; YUE = 'YUE'; FAS = 'FAS'; PER = 'PER'; JPN = 'JPN'; POL = 'POL'; RUS = 'RUS'
    POR = 'POR'; TGL = 'TGL'; ARA = 'ARA'; ITA = 'ITA'; GER = 'GER'; CRE = 'CRE'; CMN = 'CMN'
    LAO = 'LAO'; DUT = 'DUT'; THA = 'THA'; NAV = 'NAV'; ZZZ = 'ZZZ'; TAG = 'TAG'; HMG = 'HMG';
    BEN = 'BEN'; PHI = 'PHI'; MAN = 'MAN'; HIN = 'HIN'; GUJ = 'GUJ'; SRP = 'SRP'; IRA = 'IRA'
    URD = 'URD'

class RawDemographics(Base):
    __tablename__ = 'raw_demographics'

    member_id = mapped_column(ForeignKey("member.member_id"))

    bh_psyc_visit_ct_pmpm = Column(Float)
    bh_rtc_admit_ct_pmpm = Column(Float)
    bh_rtc_admit_days_pmpm = Column(Float)
    days_since_last_clm = Column(Integer)
    nonpar_allowed_pmpm_cost = Column(Float)
    riskarr_downside = Column(Boolean)
    riskarr_global = Column(Boolean)
    riskarr_rewards = Column(Boolean)
    riskarr_upside = Column(Boolean)
    rucc_category = Column(Enum(ERUCC))
    lang_spoken_cd = Column(Enum(ELangSpoken))

ArgumentError: Mapper Mapper[RawDemographics(raw_demographics)] could not assemble any primary key columns for mapped table 'raw_demographics'

In [284]:
for var in dir(ERUCC):
    if isinstance(var, str):
        print(var)

Metro_1
Metro_2
Metro_3
Metro_4
Nonemtro_9
Nonmetro_5
Nonmetro_6
Nonmetro_7
Nonmetro_8
__class__
__doc__
__members__
__module__


In [278]:
for var in vars(ERUCC):
    print(var)

_generate_next_value_
__module__
__doc__
_member_names_
_member_map_
_member_type_
_value2member_map_
Metro_1
Metro_2
Metro_3
Metro_4
Nonmetro_5
Nonmetro_6
Nonmetro_7
Nonmetro_8
Nonemtro_9
__new__


In [286]:
getattr(ERUCC, 'Metro_1')

<ERUCC.Metro_1: '1-Metro'>

In [283]:
print(getattr(ERUCC, 'Metro_1').name)
print(getattr(ERUCC, 'Metro_1').value)

Metro_1
1-Metro


## Pharmacy Utilization

In [108]:
df = dataframes['Pharmacy Utilization']
df

,rx_overall_pmpm_cost,rx_overall_net_paid_pmpm_cost,rx_overall_coins_pmpm_cost,rx_overall_copay_pmpm_cost,rx_overall_deduct_pmpm_cost,rx_overall_mbr_resp_pmpm_cost,rx_overall_gpi_pmpm_ct,rx_overall_dist_gpi6_pmpm_ct,rx_overall_pmpm_ct,rx_tier_1_pmpm_ct,rx_tier_2_pmpm_ct,rx_tier_3_pmpm_ct,rx_tier_4_pmpm_ct,rx_days_since_last_script,rx_pharmacies_pmpm_ct,rx_perphy_pmpm_ct,id,type
0,1369.120000,1366.335000,2.785000,0.000000,0.00,2.785000,11.500000,8.500000,15.000000,4.000000,3.500000,3.500000,0.500000,7.0,1.000000,0.0,1352611,Training
1,44.848330,16.679167,0.000000,28.169167,0.00,28.169167,3.166667,1.166667,3.166667,1.583333,1.166667,0.250000,0.166667,11.0,0.166667,0.0,299881,Training
2,2.383333,0.716667,0.000000,1.666667,0.00,1.666667,2.250000,0.500000,2.250000,1.916667,0.333333,0.000000,0.000000,12.0,0.166667,0.0,1191826,Training
3,707.318360,675.651700,0.000000,23.750000,0.00,31.666666,1.166667,0.500000,1.166667,0.666667,0.166667,0.333333,0.000000,26.0,0.083333,0.0,1772108,Training
4,44.096670,15.378333,0.066667,3.063333,40.00,28.718334,3.083333,1.250000,3.083333,3.083333,0.000000,0.000000,0.000000,50.0,0.083333,0.0,171691,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381971,1244.440000,1228.340000,0.000000,16.100000,388.36,16.100000,11.000000,11.000000,11.000000,3.000000,2.000000,5.000000,1.000000,8.0,2.000000,0.0,914377,Holdout
381972,21.780000,4.400000,0.000000,4.000000,13.38,17.380000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,4.0,1.000000,0.0,885778,Holdout
381973,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,480.0,0.000000,0.0,374571,Holdout
381974,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,480.0,0.000000,0.0,753477,Holdout


In [ ]:
class RawPharmacyUtilization(Base):
    __tablename__ = 'raw_pharmacy_utilization'

    member_id = mapped_column(ForeignKey("member.member_id"))

    rx_days_since_last_script = Column(Integer)
    rx_overall_coins_pmpm_cost = Column(Float)
    rx_overall_copay_pmpm_cost = Column(Float)
    rx_overall_deduct_pmpm_cost = Column(Float)
    rx_overall_dist_gpi6_pmpm_ct = Column(Float)
    rx_overall_gpi_pmpm_ct = Column(Float)
    rx_overall_mbr_resp_pmpm_cost = Column(Float)
    rx_overall_net_paid_pmpm_cost = Column(Float)
    rx_overall_pmpm_cost = Column(Float)
    rx_overall_pmpm_ct = Column(Float)
    rx_perphy_pmpm_ct = Column(Float)
    rx_pharmacies_pmpm_ct = Column(Float)
    rx_tier_1_pmpm_ct = Column(Float)
    rx_tier_2_pmpm_ct = Column(Float)
    rx_tier_3_pmpm_ct = Column(Float)
    rx_tier_4_pmpm_ct = Column(Float)

## Sales Channel

In [111]:
df = dataframes['Sales Channel']
df

,channel,id,type
0,Field,1069625,Training
1,Field,326883,Training
2,Consumer Direct,1559370,Training
3,Consumer Direct,13259,Training
4,Field,1254048,Training
...,...,...,...
381971,Consumer Direct,1420682,Holdout
381972,Consumer Direct,126591,Holdout
381973,Consumer Direct,655161,Holdout
381974,Partner Call Center,1128703,Holdout


In [112]:
df['channel'].unique()

array(['Field', 'Consumer Direct', 'DMS Telesales', 'Partner Call Center',
       'Brokerage', nan], dtype=object)

In [ ]:
class EChannel(enum.Enum):
    Field = 'Field'
    ConsumerDirect = 'Consumer Direct'
    DMSTelesales = 'DMS Telesales'
    PartnerCallCenter = 'Partner Call Center'
    Brokerage = 'Brokerage'

class RawChannel(Base):
    __tablename__ = 'raw_channel'

    member_id = mapped_column(ForeignKey("member.member_id"))
    channel = Column(Enum(EChannel))


## Social Determinants of Health

In [114]:
df = dataframes['Social Determinants of Health']
df

,rwjf_premature_death_rate,rwjf_poor_health_pct,rwjf_uninsured_adults_pct,rwjf_pcp_rate,rwjf_preventable_ip_rate,rwjf_adult_smoking_pct,rwjf_adult_obesity_pct,rwjf_teen_births_rate,rwjf_homicides_rate,rwjf_high_school_pct,...,rwjf_home_ownership_pct,rwjf_housing_cost_burden_pct,rwjf_flu_vax,rwjf_suicides_rate,rwjf_broadband_access,rwjf_diabetes_monitor_pct,rwjf_healthcare_cost,rwjf_drug_deaths_modl_rate,id,type
0,8671.084,0.216076,0.197464,0.000265,52.68,0.195525,0.398,24.084312,10.428421,0.870000,...,0.676266,0.159998,0.49,14.443182,0.855341,0.863121,9750.99,6.0,1352611,Training
1,7439.620,0.193420,0.072883,0.000603,32.74,0.196184,0.311,14.443236,NaN,0.970000,...,0.707001,0.109818,0.55,24.068380,0.833414,0.881720,8749.57,11.0,299881,Training
2,8363.172,0.211875,0.090306,0.000797,33.49,0.218531,0.348,22.917480,3.817668,0.940000,...,0.645821,0.120681,0.56,14.110594,0.840763,0.896078,9544.74,12.0,1191826,Training
3,10805.334,0.281196,0.092180,0.000284,55.07,0.281043,0.330,45.366043,6.421520,0.930000,...,0.783335,0.081525,0.37,16.813110,0.685470,0.890449,10177.70,12.0,1772108,Training
4,14181.182,0.313580,0.088424,0.000586,107.71,0.289167,0.407,57.787067,6.553408,0.970000,...,0.699067,0.147324,0.34,19.058668,0.719067,0.821641,13539.30,15.0,171691,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381971,7365.378,0.157742,0.172277,0.001064,26.52,0.169902,0.295,21.497534,6.127874,0.842728,...,0.674844,0.107339,0.55,15.421494,0.833155,0.866744,9025.13,10.0,914377,Holdout
381972,5351.338,0.153297,0.086778,0.000506,29.67,0.129318,0.303,12.169574,2.361818,0.908594,...,0.670935,0.131618,0.50,14.466125,0.908462,0.888097,7993.94,10.0,885778,Holdout
381973,7027.115,0.169935,0.072991,0.000703,42.95,0.205967,0.308,19.377674,4.437670,0.877775,...,0.748801,0.111504,0.59,17.103693,0.816211,0.899408,9735.85,8.0,374571,Holdout
381974,5840.262,0.159544,0.111326,0.000731,44.87,0.177218,0.280,13.767016,4.436833,0.900000,...,0.637098,0.131638,0.48,12.705891,0.898676,0.854811,9750.53,6.0,753477,Holdout


In [115]:
df['rwjf_drinkwater_violate_ind'].unique()

array([ 1.,  0., nan])

In [ ]:
class RawSocialDeterminantsOfHealth(Base):
    __tablename__ = 'raw_social_determinants_of_health'

    member_id = mapped_column(ForeignKey("member.member_id"))
    rwjf_preventable_ip_rate = Column(Float)
    rwjf_healthcare_cost = Column(Float)
    rwjf_other_pcp = Column(Float)
    rwjf_uninsured_adults_pct = Column(Float)
    rwjf_uninsured_child_pct = Column(Float)
    rwjf_diabetes_monitor_pct = Column(Float)
    rwjf_flu_vax = Column(Float)
    rwjf_mammography_pct = Column(Float)
    rwjf_uninsured_pct = Column(Float)
    rwjf_pcp_rate = Column(Float)
    rwjf_dentists_ratio = Column(Float)
    rwjf_men_hlth_prov_ratio = Column(Float)
    rwjf_age_gt_65_pct = Column(Float)
    rwjf_native_race_pct = Column(Float)
    rwjf_asian_race_pct = Column(Float)
    rwjf_age_lt_18_pct = Column(Float)
    rwjf_female_pct = Column(Float)
    rwjf_hispanic_pct = Column(Float)
    rwjf_hawaiian_race_pct = Column(Float)
    rwjf_african_race_pct = Column(Float)
    rwjf_white_race_pct = Column(Float)
    rwjf_non_english_pct = Column(Float)
    rwjf_rural_pct = Column(Float)
    rwjf_population = Column(Float)
    rwjf_drug_overdose_deaths_rate = Column(Float)
    rwjf_drug_deaths_modl_rate = Column(Float)
    rwjf_food_insecurity_pct = Column(Float)
    rwjf_food_env_inx = Column(Float)
    rwjf_insufficient_sleep_pct = Column(Float)
    rwjf_limit_hlthy_food_pct = Column(Float)
    rwjf_mv_deaths_rate = Column(Float)
    rwjf_teen_births_rate = Column(Float)
    rwjf_std_infect_rate = Column(Float)
    rwjf_inactivity_pct = Column(Float)
    rwjf_alcoholic_pct = Column(Float)
    rwjf_adult_obesity_pct = Column(Float)
    rwjf_adult_smoking_pct = Column(Float)
    rwjf_dui_deaths_pct = Column(Float)
    rwjf_exercise_access_pct = Column(Float)
    rwjf_mental_distress_pct = Column(Float)
    rwjf_physical_distress_pct = Column(Float)
    rwjf_premature_death_rate = Column(Float)
    rwjf_poor_men_hlth_days = Column(Float)
    rwjf_poor_phy_hlth_days = Column(Float)
    rwjf_life_expectancy = Column(Float)
    rwjf_child_mortality = Column(Float)
    rwjf_diabetes_pct = Column(Float)
    rwjf_hiv_rate = Column(Float)
    rwjf_infant_mortality = Column(Float)
    rwjf_poor_health_pct = Column(Float)
    rwjf_low_birthweight_pct = Column(Float)
    rwjf_premature_mortality = Column(Float)
    rwjf_long_commute_alone_pct = Column(Float)
    rwjf_air_pollute_density = Column(Float)
    rwjf_drinkwater_violate_ind = Column(Boolean)
    rwjf_housing_cost_burden_pct = Column(Float)
    rwjf_severe_housing_pct = Column(Float)
    rwjf_broadband_access = Column(Float)
    rwjf_home_ownership_pct = Column(Float)
    rwjf_drive_alone_pct = Column(Float)
    rwjf_disconnect_youth_pct = Column(Float)
    rwjf_child_free_lunch_pct = Column(Float)
    rwjf_firearm_fatalities_rate = Column(Float)
    rwjf_homicides_rate = Column(Float)
    rwjf_median_house_income = Column(Float)
    rwjf_injury_deaths_rate = Column(Float)
    rwjf_social_associate_rate = Column(Float)
    rwjf_violent_crime_rate = Column(Float)
    rwjf_some_college_pct = Column(Float)
    rwjf_single_parent_pct = Column(Float)
    rwjf_child_poverty_pct = Column(Float)
    rwjf_high_school_pct = Column(Float)
    rwjf_unemploy_pct = Column(Float)
    rwjf_income_inequ_ratio = Column(Float)
    rwjf_resident_seg_black_inx = Column(Float)
    rwjf_resident_seg_nonwhite_inx = Column(Float)
    rwjf_suicides_rate = Column(Float)

## Web Activity

In [119]:
df = dataframes['Web Activity']
df

,days_since_last_login,login_pmpm_ct,login_count_0,login_count_1,login_count_2,login_count_3,login_count_4,login_count_5,login_count_6,login_count_7,login_count_8,login_count_9,login_count_10,login_count_11,id,type
0,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1352611,Training
1,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,299881,Training
2,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1191826,Training
3,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1772108,Training
4,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,171691,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381971,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,914377,Holdout
381972,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,885778,Holdout
381973,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,374571,Holdout
381974,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,753477,Holdout


In [ ]:
class RawWebActivity(Base):
    __tablename__ = 'raw_web_activity'

    member_id = mapped_column(ForeignKey("member.member_id"))
    login_count_0 = Column(Integer)
    login_count_1 = Column(Integer)
    login_count_10 = Column(Integer)
    login_count_11 = Column(Integer)
    login_count_2 = Column(Integer)
    login_count_3 = Column(Integer)
    login_count_4 = Column(Integer)
    login_count_5 = Column(Integer)
    login_count_6 = Column(Integer)
    login_count_7 = Column(Integer)
    login_count_8 = Column(Integer)
    login_count_9 = Column(Integer)
    login_pmpm_ct = Column(Float)
    days_since_last_login = Column(Integer)


## Member Data

In [120]:
df = dataframes['MEMBER_DATA']
df

,consec_tenure_month,all_mm_tenure,tenure_band,dual_eligible_ind,disabled_ind,lis_ind,id,type
0,6,6,0 - 0.5 YEARS,N,N,N,1551235,Training
1,59,59,4 - 5 YEARS,N,Y,N,1643841,Training
2,119,119,7+ YEARS,N,N,N,540296,Training
3,24,24,1.5 - 2 YEARS,N,Y,N,729600,Training
4,60,76,4 - 5 YEARS,N,N,N,716836,Training
...,...,...,...,...,...,...,...,...
381971,57,57,4 - 5 YEARS,N,N,N,767776,Holdout
381972,39,39,3 - 4 YEARS,N,N,N,120431,Holdout
381973,36,36,2 - 3 YEARS,N,N,N,1705262,Holdout
381974,1,1,0 - 0.5 YEARS,N,N,N,925014,Holdout


In [121]:
df['consec_tenure_month']

0           6
1          59
2         119
3          24
4          60
         ... 
381971     57
381972     39
381973     36
381974      1
381975      2
Name: consec_tenure_month, Length: 1909880, dtype: int64

In [122]:
df['all_mm_tenure']

0           6
1          59
2         119
3          24
4          76
         ... 
381971     57
381972     39
381973     36
381974      1
381975      2
Name: all_mm_tenure, Length: 1909880, dtype: int64

In [124]:
df['tenure_band'].unique()

array(['0 - 0.5 YEARS', '4 - 5 YEARS', '7+ YEARS', '1.5 - 2 YEARS',
       '3 - 4 YEARS', '2 - 3 YEARS', '0.5 - 1 YEARS', '6 - 7 YEARS',
       '5 - 6 YEARS', '1 - 1.5 YEARS'], dtype=object)

In [126]:
df['dual_eligible_ind'].unique()

array(['N', 'Y'], dtype=object)

In [127]:
df['disabled_ind'].unique()

array(['N', 'Y'], dtype=object)

In [128]:
df['lis_ind'].unique()

array(['N', 'Y'], dtype=object)

In [ ]:
class ETenureBand(enum.Enum):
    _0_0p5 = '0 - 0.5 YEARS'
    _0p5_1 = '0.5 - 1 YEARS'
    _1_1p5 = '1 - 1.5 YEARS'
    _1p5_2 = '1.5 - 2 YEARS'
    _2_3 = '2 - 3 YEARS'
    _3_4 = '3 - 4 YEARS'
    _4_5 = '4 - 5 YEARS'
    _5_6 = '5 - 6 YEARS'
    _6_7 = '6 - 7 YEARS'
    _7plus = '7+ YEARS'

class RawMemberData(Base):
    __tablename__ = 'raw_member_data'

    member_id = mapped_column(ForeignKey("member.member_id"))
    consec_tenure_month = Column(Integer)
    all_mm_tenure = Column(Integer)
    tenure_band = Column(Enum(ETenureBand))
    dual_eligible_ind = Column(Boolean)
    disabled_ind = Column(Boolean)
    lis_ind = Column(Boolean)

## Quality data

In [132]:
df = dataframes['QUALITY_DATA']
df

,measurement_year,measure_name,measure_desc,measure_type,base_event_date,compliant_cnt,eligible_cnt,id,type
0,2020,ADH (DIAB),Medication Adherence for Diabetes Medications,Patient Safety,NaN,1.0,1,40240,Training
1,2020,ADH (ACE),Medication Adherence for Hypertension (RAS ant...,Patient Safety,NaN,1.0,1,40240,Training
2,2020,ABA,Adult BMI Assessment,HEDIS,NaN,0.0,1,193657,Training
3,2020,ADH (ACE),Medication Adherence for Hypertension (RAS ant...,Patient Safety,NaN,0.0,1,1685072,Training
4,2020,ADH (STATIN),Medication Adherence for Cholesterol (Statins),Patient Safety,NaN,1.0,1,1685072,Training
...,...,...,...,...,...,...,...,...,...
8377981,2020,TEX,Dr Talks Mental Health,Patient Experience,30JUL2020,1.0,1,1062632,Holdout
8377982,2020,TEX,Dr Talks Mental Health,Patient Experience,22OCT2020,0.0,1,1425434,Holdout
8377983,2020,TEX,Dr Talks Mental Health,Patient Experience,12MAY2020,1.0,1,1104218,Holdout
8377984,2021,TEX,Dr Talks Mental Health,Patient Experience,09APR2021,1.0,1,669409,Holdout


In [143]:
df['measurement_year'].unique()

array([2020, 2021, 2022])

In [134]:
df['measure_name'].unique()

array(['ADH (DIAB)', 'ADH (ACE)', 'ABA', 'ADH (STATIN)', 'PCR',
       'TRC (MRP)', 'CBP', 'COL', 'CDC (EYE)', 'CDC (NPH)', 'CDC (HbA1c)',
       'BCS', 'SUPD', 'SPC STATIN', 'ART', 'OMW', 'COA (MDR)',
       'COA (PNS)', 'COA (FSA)', 'COL (45-50)', 'EED', 'FMC', 'HBD',
       'KED', 'MRP', 'TRC (PED)', 'ESA', 'SWT', 'EGR', 'RXC', 'ETA',
       'MDR', 'ASV', 'TFP', 'SFT', 'TBC', 'SBT', 'TEX'], dtype=object)

In [136]:
df['measure_desc'].unique()

array(['Medication Adherence for Diabetes Medications',
       'Medication Adherence for Hypertension (RAS antagonists)',
       'Adult BMI Assessment',
       'Medication Adherence for Cholesterol (Statins)',
       'Plan All-Cause Readmissions*',
       'Transitions of Care - Medication Reconciliation Post-Discharge',
       'Controlling High Blood Pressure', 'Colorectal Cancer Screening',
       'Diabetes Care - Eye Exam',
       'Diabetes Care - Kidney Disease Monitoring',
       'Diabetes Care - Blood Sugar Controlled',
       'Breast Cancer Screening', 'Statin Use in Persons with Diabetes',
       'Statin Therapy for Patients With Cardiovascular Disease',
       'Rheumatoid Arthritis Management',
       'Osteoporosis Management in Women who had a Fracture',
       'Care for Older Adults - Medication Review',
       'Care for Older Adults - Pain Screening',
       'Care for Older Adults - Functional Status Assessment',
       'Comprehensive Diabetes Care - Medical Attention for Ne

In [137]:
df['measure_type'].unique()

array(['Patient Safety', 'HEDIS', 'Patient Experience'], dtype=object)

In [138]:
df['base_event_date']

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
             ...    
8377981    30JUL2020
8377982    22OCT2020
8377983    12MAY2020
8377984    09APR2021
8377985    27MAY2020
Name: base_event_date, Length: 41950227, dtype: object

In [140]:
df['compliant_cnt'].unique()

array([1.  , 0.  , 0.33, 0.66])

In [142]:
df['eligible_cnt'].unique()

array([1])

In [ ]:
class EMeasureName(enum.Enum):
    ADH_DIAB = 'ADH (DIAB)'; ADH_ACE = 'ADH (ACE)'; ABA = 'ABA'; ADH_STATIN = 'ADH (STATIN)'
    PCR = 'PCR'; TRC_MRP = 'TRC (MRP)'; CBP = 'CBP'; COL = 'COL'; CDC_EYE = 'CDC (EYE)'
    CDC_NPH = 'CDC (NPH)'; CHC_HbA1c = 'CDC (HbA1c)'; BCS = 'BCS'; SUPD = 'SUPD'
    SPC_STATIN = 'SPC STATIN'' ART = 'ART'; OMW = 'OMW'; COA_MDR = 'COA (MDR)'
    COA_PNS = 'COA (PNS)'; COA_FSA = 'COA (FSA)'; COL_45_50 = 'COL (45-50)'; EED = 'EED'
    FMC = 'FMC'; HBD = 'HBD'; KED = 'KED'; MRP = 'MRP'; TRC_PED = 'TRC (PED)'; ESA = 'ESA'
    SWT = 'SWT'; EGR = 'EGR'; RXC = 'RXC'; ETA = 'ETA'; MDR = 'MDR'; ASV = 'ASV'; TFP = 'TFP'
    SFT = 'SFT'; TBC = 'TBC'; SBT = 'SBT'; TEX = 'TEX'

class EMeasureType(enum.Enum):
    PatientSafety = 'Patient Safety'
    HEDIS = 'HEDIS'
    PatientExperience = 'Patient Experience'

class RawQualityData(Base):
    __tablename__ = 'raw_quality_data'

    member_id = mapped_column(ForeignKey("member.member_id"))
    measurement_year = Column(Integer)
    measure_name = Column(Enum(EMeasureName))
    measure_desc = Column(Text)
    measure_type = Column(Enum(EMeasureType))
    base_event_date = Column(Date)
    compliant_cnt = Column(Float)
    eligible_cnt = Column(Integer)

## Member Details

In [145]:
df = dataframes['humana_mays_target_member_details']
df

,generic_grouper,unattributed_provider,sex_cd,age,veteran_ind,mco_contract_nbr,plan_benefit_package_id,pbp_segment_id,state_of_residence,county_of_residence,region,race,id,type
0,Y,N,F,74,N,H5216,72,0,FL,POLK,Florida,NaN,1081255,Training
1,N,N,M,83,N,H5216,334,0,MS,HINDS,Gulf South,BLACK,9020,Training
2,Y,Y,F,68,N,H5216,43,1,TX,GRAYSON,South Central,N AMERICAN NATIVE,1089029,Training
3,N,N,F,66,N,H5216,805,0,AL,HOUSTON,Gulf South,WHITE,872819,Training
4,Y,N,M,74,N,H5216,805,0,TN,SUMNER,Gulf South,WHITE,830574,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381971,Y,Y,M,67,N,H5216,205,0,GA,WHITFIELD,Southeast,WHITE,304443,Holdout
381972,Y,Y,M,78,N,H5216,805,0,NC,MECKLENBURG,Southeast,WHITE,1766546,Holdout
381973,N,N,M,83,N,H5216,323,0,OK,OKLAHOMA,South Central,NaN,452566,Holdout
381974,N,N,F,84,N,H0473,3,0,TX,WILLIAMSON,South Central,NaN,1600005,Holdout


In [146]:
df['generic_grouper'].unique()

array(['Y', 'N', nan], dtype=object)

In [147]:
df['unattributed_provider'].unique()

array(['N', 'Y', nan], dtype=object)

In [148]:
df['sex_cd'].unique()

array(['F', 'M', 'U'], dtype=object)

In [150]:
df['age']

0         74
1         83
2         68
3         66
4         74
          ..
381971    67
381972    78
381973    83
381974    84
381975    82
Name: age, Length: 1909880, dtype: int64

In [151]:
df['veteran_ind'].unique()

array(['N', 'Y'], dtype=object)

In [153]:
df['mco_contract_nbr'].unique()

array(['H5216', 'H8087', 'H5970', 'H5525', 'H9070', 'H7284', 'H0473',
       'H7617', 'H2029', 'H6622', 'H1036'], dtype=object)

In [154]:
df['plan_benefit_package_id'].unique()

array([ 72, 334,  43, 805,  19, 266,   1, 806, 349, 169, 247, 207, 203,
       258, 317, 188,  70,  15, 304, 214, 809, 136,  21, 144, 286,  24,
       318, 303, 301, 312,  35,   3, 360, 233, 190, 251, 199, 231,  63,
       348,  23, 329,  68, 279, 138, 182, 321, 200, 205,   4, 124,   8,
        16,  51, 347, 131, 254, 267, 171, 175, 285,   6, 327, 196,  36,
       337, 232, 154, 228, 128, 211,  41, 137,   9,   5, 197,  83, 298,
       218, 224, 236, 309, 213, 206, 275, 255, 253, 216, 311,  27, 220,
       343, 225, 142,  50, 306, 192, 283, 176,  81, 280,  64, 358, 359,
       352, 244, 212, 201, 221, 339, 186,  53, 328, 249, 278, 310,  18,
       223,  62, 120, 801, 226, 103,  13, 160,  74, 229,  89, 167, 300,
       322, 269, 114,  44, 273, 106,  34, 194,  17, 323, 217,  42,  20,
       164, 284, 302,  29, 252,  58, 263, 170, 355, 265, 152, 177, 133,
         2, 287,  73, 331,  11, 292,  55, 268, 116,  49,  22, 215,  10,
       140,  65,  96,  46,  78, 179, 235, 332,  92, 195, 260,  5

In [155]:
df['pbp_segment_id'].unique()

array([0, 1, 2, 3, 6, 4, 5])

In [156]:
df['state_of_residence'].unique()

array(['FL', 'MS', 'TX', 'AL', 'TN', 'NC', 'KY', 'VA', 'MI', 'NY', 'GA',
       'NJ', 'WA', 'WI', 'WV', 'KS', 'MN', 'IL', 'HI', 'SC', 'NM', 'AR',
       'OH', 'OK', 'MA', 'OR', 'MT', 'PA', 'UT', 'IA', 'CO', 'ME', 'LA',
       'AZ', 'IN', 'NV', 'ID', 'SD', 'ND', 'MO', 'MD', 'NH', 'AK', 'NE',
       'CA', 'PR', 'CT', 'WY', 'VT', 'RI', 'DE', 'DC', 'VI', 'GU', 'MP',
       'AS'], dtype=object)

In [157]:
df['county_of_residence'].unique()

array(['POLK', 'HINDS', 'GRAYSON', ..., 'PLUMAS', 'ALEUTIANS WEST',
       'FRONTIER'], dtype=object)

In [158]:
df[df['county_of_residence'] == 'PLUMAS']

,generic_grouper,unattributed_provider,sex_cd,age,veteran_ind,mco_contract_nbr,plan_benefit_package_id,pbp_segment_id,state_of_residence,county_of_residence,region,race,id,type
329482,Y,Y,F,65,N,H5216,805,0,CA,PLUMAS,Pacific Southwest,NaN,631450,Holdout


In [160]:
df['race'].unique()

array([nan, 'BLACK', 'N AMERICAN NATIVE', 'WHITE', 'ASIAN', 'HISPANIC',
       'UNKNOWN', 'OTHER'], dtype=object)

In [ ]:
class ESex(enum.Enum):
    F = 'F'
    M = 'M'
    U = 'U'

class EMcoContractNbr(enum.Enum):
    H5216 = 'H5216'; H8087 = 'H8087'; H5970 = 'H5970'; H5525 = 'H5525'; H9070 = 'H9070'
    H7284 = 'H7284'; H0473 = 'H0473'; H7617 = 'H7617'; H2029 = 'H2029'; H6622 = 'H6622'; H1036 = 'H1036'

class EState(enum.Enum):
    FL = 'FL'; MS = 'MS'; TX = 'TX'; AL = 'AL'; TN = 'TN'; NC = 'NC'; KY = 'KY'; VA = 'VA'
    MI = 'MI'; NY = 'NY'; GA = 'GA'; NJ = 'NJ'; WA = 'WA'; WI = 'WI'; WV = 'WV'; KS = 'KS'
    MN = 'MN'; IL = 'IL'; HI = 'HI'; SC = 'SC'; NM = 'NM'; AR = 'AR'; OH = 'OH'; OK = 'OK'
    MA = 'MA'; OR = 'OR'; MT = 'MT'; PA = 'PA'; UT = 'UT'; IA = 'IA'; CO = 'CO'; ME = 'ME'
    LA = 'LA'; AZ = 'AZ'; IN = 'IN'; NV = 'NV'; ID = 'ID'; SD = 'SD'; ND = 'ND'; MO = 'MO'
    MD = 'MD'; NH = 'NH'; AK = 'AK'; NE = 'NE'; CA = 'CA'; PR = 'PR'; CT = 'CT'; WY = 'WY'
    VT = 'VT'; RI = 'RI'; DE = 'DE'; DC = 'DC'; VI = 'VI'; GU = 'GU'; MP = 'MP'; AS = 'AS'

class ERace(enum.Enum):
    Black = 'BLACK'
    NAmericanNative = 'N AMERICAN NATIVE'
    White = 'WHITE'
    Asian = 'ASIAN'
    Hispanice = 'HISPANIC'
    Unknown = 'UNKNOWN'
    Other = 'OTHER'

class RawMemberDetails(Base):
    __tablename__ = 'raw_member_details'

    member_id = mapped_column(ForeignKey("member.member_id"))
    generic_grouper = Column(Boolean)
    unattributed_provider = Column(Boolean)
    sex_cd = Column(Enum(ESex))
    age = Column(Integer)
    veteran_ind = Column(Boolean)
    mco_contract_nbr = Column(Enum(EMcoContractNbr))
    plan_benefit_package_id = Column(Integer)
    state_of_residence = Column(Enum(EState))
    county_of_residence = Column(Text)
    race = Column(Enum(ERace))

## Member Claims

In [162]:
df = dataframes['humana_mays_target_member_visit_claims']
df

,dos_year,clm_unique_key,serv_date_skey,pcp_visit,annual_wellness,humana_paf,preventative_visit,comp_physical_exam,ihwa,fqhc_visit,...,cardiologist_visit,gastroenterologist_visit,orthopedist_visit,obgyn_visit,nephroloogist_visit,pulmonologist_visit,urgent_care_visit,er_visit,id,type
0,2021,90000007336,20210105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,733133,Training
1,2021,90000030533,20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296887,Training
2,2021,90000039180,20210111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1331806,Training
3,2021,90000048032,20210112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1548280,Training
4,2021,90000070091,20210118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,689644,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4857271,2022,999789000483222003,20220613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,295810,Holdout
4857272,2021,999917100480152103,20210203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,1825873,Holdout
4857273,2022,999946000482932203,20220516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,957758,Holdout
4857274,2021,999956100479932203,20210706,Y,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,1379188,Holdout


In [168]:
df['pcp_visit'].unique()

array([nan, 'Y'], dtype=object)

In [169]:
df['annual_wellness'].unique()

array([nan, 'Y'], dtype=object)

In [170]:
df['humana_paf'].unique()

array([nan, 'Y'], dtype=object)

In [171]:
df['preventative_visit'].unique()

array([nan, 'Y'], dtype=object)

In [173]:
df['comp_physical_exam'].unique()

array([nan, 'Y'], dtype=object)

In [174]:
df['ihwa'].unique()

array([nan])

In [176]:
df['fqhc_visit'].unique()

array([nan, 'Y'], dtype=object)

In [177]:
df['telehealth'].unique()

array([nan, 'Y'], dtype=object)

In [179]:
df['endocrinologist_visit'].unique()

array([nan, 'Y'], dtype=object)

In [180]:
df['oncolologist_visit'].unique()

array([nan, 'Y'], dtype=object)

In [181]:
df['radiologist_visit'].unique()

array([nan, 'Y'], dtype=object)

In [ ]:
class RawMemberClaims(Base):
    __tablename__ = 'raw_member_claims'

    member_id = mapped_column(ForeignKey("member.member_id"))
    dos_year = Column(Integer)
    clm_unique_key = Column(Text)
    serv_date_skey = Column(Date)
    pcp_visit = Column(Boolean)
    annual_wellness = Column(Boolean)
    humana_paf = Column(Boolean)
    preventative_visit = Column(Boolean)
    comp_physical_exam = Column(Boolean)
    ihwa = Column(Boolean)
    fqhc_visit = Column(Boolean)
    telehealth = Column(Boolean)
    endocrinologist_visit = Column(Boolean)
    oncolologist_visit = Column(Boolean)
    radiologist_visit = Column(Boolean)
    podiatrist_visit = Column(Boolean)
    ophthalmologist_visit = Column(Boolean)
    optometrist_visit = Column(Boolean)
    physical_therapist_visit = Column(Boolean)
    cardiologist_visit = Column(Boolean)
    gastroenterologist_visit = Column(Boolean)
    orthopedist_visit = Column(Boolean)
    obgyn_visit = Column(Boolean)
    nephroloogist_visit = Column(Boolean)
    pulmonologist_visit = Column(Boolean)
    urgent_care_visit = Column(Boolean)
    er_visit = Column(Boolean)

## Member Condition

In [183]:
df = dataframes['humana_mays_target_member_conditions']
df

,cond_key,chronicity,cond_desc,hcc_model_type,cms_model_vers_cd,membership_year,id,type
0,226,Chronic,"Heart Failure, Except End-Stage and Acute",MEDICAL,V28,2023,993642,Training
1,94,Chronic,Systemic Lupus Erythematosus and Other Specifi...,MEDICAL,V28,2023,993642,Training
2,329,Chronic,"Chronic Kidney Disease, Moderate Stage 3, Exce...",MEDICAL,V28,2023,993642,Training
3,81,Chronic,Ulcerative Colitis,MEDICAL,V28,2023,803124,Training
4,47,Chronic,Disorders of Immunity,MEDICAL,V24,2023,803124,Training
...,...,...,...,...,...,...,...,...
1001767,22,Chronic,"Bladder, Colorectal, and Other Cancers",MEDICAL,V28,2023,1631128,Holdout
1001768,48,Chronic,Morbid Obesity,MEDICAL,V28,2023,787802,Holdout
1001769,38,Chronic,"Diabetes with Glycemic, Unspecified, or No Com...",MEDICAL,V28,2023,787802,Holdout
1001770,23,Chronic,"Prostate, Breast, and Other Cancers and Tumors",MEDICAL,V28,2023,365181,Holdout


In [184]:
df['cond_key']

0          226
1           94
2          329
3           81
4           47
          ... 
1001767     22
1001768     48
1001769     38
1001770     23
1001771     59
Name: cond_key, Length: 5011114, dtype: int64

In [185]:
df['chronicity'].unique()

array(['Chronic'], dtype=object)

In [186]:
df['cond_desc']

0                  Heart Failure, Except End-Stage and Acute
1          Systemic Lupus Erythematosus and Other Specifi...
2          Chronic Kidney Disease, Moderate Stage 3, Exce...
3                                         Ulcerative Colitis
4                                      Disorders of Immunity
                                 ...                        
1001767               Bladder, Colorectal, and Other Cancers
1001768                                       Morbid Obesity
1001769    Diabetes with Glycemic, Unspecified, or No Com...
1001770       Prostate, Breast, and Other Cancers and Tumors
1001771    Major Depressive, Bipolar, and Paranoid Disorders
Name: cond_desc, Length: 5011114, dtype: object

In [187]:
df['hcc_model_type'].unique()

array(['MEDICAL', 'ESRD'], dtype=object)

In [188]:
df['cms_model_vers_cd'].unique()

array(['V28', 'V24'], dtype=object)

In [189]:
df['membership_year']

0          2023
1          2023
2          2023
3          2023
4          2023
           ... 
1001767    2023
1001768    2023
1001769    2023
1001770    2023
1001771    2023
Name: membership_year, Length: 5011114, dtype: int64

In [ ]:
class EChronicity(enum.Enum):
    Chronic = 'Chronic'

class EHccModelType(enum.Enum):
    Medical = 'MEDICAL'
    ESRD = 'ESRD'

class ECMSModelVers(enum.Enum):
    V28 = 'V28'
    V24 = 'V24'

class RawMemberCondition(Base):
    __tablename__ = 'raw_member_condition'

    member_id = mapped_column(ForeignKey("member.member_id"))
    cond_key = Column(Integer)
    chronicity = Column(Enum(EChronicity))
    cond_desc = Column(Text)
    hcc_model_type = Column(Enum(EHccModelType))
    cms_model_vers_cd = Column(Enum(ECMSModelVers))
    membership_year = Column(Integer)

## Merge attempt 1 (Failed due to OOM)

In [8]:
merged_df = None
ncols = 0
for icsv, (csv_name, df) in enumerate(dataframes.items()):
    print("[{}/{}] Merging csv {}".format(icsv+1, len(csv_files), csv_name))
    print("Cols (#={}): {}".format(len(df.columns), df.columns))
    print("#rows: {}".format(len(df)))
    df = df.head()
    assert 'id' in df.columns
    ncols += len(df.columns) - 1
    if merged_df is None:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on='id', how='inner')
    print("#cols in merged: {}".format(len(merged_df.columns)))
    print("#cols total: {}".format(ncols))

[1/14] Merging csv Pharmacy Utilization_Holdout
Cols (#=17): Index(['rx_overall_pmpm_cost', 'rx_overall_net_paid_pmpm_cost',
       'rx_overall_coins_pmpm_cost', 'rx_overall_copay_pmpm_cost',
       'rx_overall_deduct_pmpm_cost', 'rx_overall_mbr_resp_pmpm_cost',
       'rx_overall_gpi_pmpm_ct', 'rx_overall_dist_gpi6_pmpm_ct',
       'rx_overall_pmpm_ct', 'rx_tier_1_pmpm_ct', 'rx_tier_2_pmpm_ct',
       'rx_tier_3_pmpm_ct', 'rx_tier_4_pmpm_ct', 'rx_days_since_last_script',
       'rx_pharmacies_pmpm_ct', 'rx_perphy_pmpm_ct', 'id'],
      dtype='object')
#rows: 381976
#cols in merged: 17
#cols total: 16
[2/14] Merging csv Social Determinants of Health_Holdout
Cols (#=78): Index(['rwjf_premature_death_rate', 'rwjf_poor_health_pct',
       'rwjf_uninsured_adults_pct', 'rwjf_pcp_rate',
       'rwjf_preventable_ip_rate', 'rwjf_adult_smoking_pct',
       'rwjf_adult_obesity_pct', 'rwjf_teen_births_rate',
       'rwjf_homicides_rate', 'rwjf_high_school_pct', 'rwjf_unemploy_pct',
       'rwjf_c

# Misc

In [4]:
dump_path = Path("out/dump20241004.csv")
dump_df = pd.read_csv(dump_path)
dump_df

/tmp/ipykernel_26585/1445172219.py:2: DtypeWarning: Columns (5,11,117,118,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  dump_df = pd.read_csv(dump_path)


,id,data_type,calendar_year,product_type,plan_category,preventive_visit_gap_ind,cci_score,dcsi_score,fci_score,cms_tot_partd_payment_amt,...,generic_grouper,unattributed_provider,sex_cd,age,veteran_ind,mco_contract_nbr,plan_benefit_package_id,state_of_residence,county_of_residence,race
0,380667,EType.Training,2023,EProductType.LPPO,EPlanCategory.MedicareAdvantage,False,3.0,0.0,4.0,96.02,...,NaN,NaN,ESex.F,66,NaN,EMcoContractNbr.H5216,58,EState.NH,CARROLL,ERace.White
1,378860,EType.Training,2023,EProductType.LPPO,EPlanCategory.MedicareAdvantage,False,2.0,0.0,1.0,76.15,...,NaN,NaN,ESex.F,68,NaN,EMcoContractNbr.H5216,1,EState.WI,WAUKESHA,NaN
2,1370287,EType.Training,2023,EProductType.LPPO,EPlanCategory.MedicareAdvantage,True,2.0,0.0,0.0,143.53,...,NaN,NaN,ESex.F,67,NaN,EMcoContractNbr.H5216,317,EState.KY,LOGAN,ERace.White
3,184540,EType.Training,2023,EProductType.LPPO,EPlanCategory.MedicareAdvantage,False,12.0,3.0,6.0,99.28,...,NaN,NaN,ESex.F,81,NaN,EMcoContractNbr.H5216,805,EState.NC,GUILFORD,NaN
4,1628837,EType.Training,2023,EProductType.LPPO,EPlanCategory.MedicareAdvantage,False,3.0,0.0,6.0,138.20,...,NaN,NaN,ESex.F,79,NaN,EMcoContractNbr.H9070,6,EState.OK,OKLAHOMA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909875,1691098,EType.Holdout,2023,EProductType.LPPO,EPlanCategory.MedicareAdvantage,NaN,6.0,2.0,4.0,146.24,...,NaN,NaN,ESex.M,92,NaN,EMcoContractNbr.H5525,51,EState.PA,BUCKS,NaN
1909876,63411,EType.Holdout,2023,EProductType.LPPO,EPlanCategory.MedicareAdvantage,NaN,5.0,1.0,1.0,75.81,...,NaN,NaN,ESex.M,78,NaN,EMcoContractNbr.H5216,311,EState.FL,ST. LUCIE,ERace.Other
1909877,1464366,EType.Holdout,2023,EProductType.LPPO,EPlanCategory.MedicareAdvantage,NaN,2.0,1.0,1.0,271.70,...,NaN,NaN,ESex.M,70,NaN,EMcoContractNbr.H5970,24,EState.NY,NEW YORK,ERace.Other
1909878,715653,EType.Holdout,2023,EProductType.LPPO,EPlanCategory.MedicareAdvantage,NaN,3.0,0.0,2.0,NaN,...,NaN,NaN,ESex.F,63,NaN,EMcoContractNbr.H5216,318,EState.KS,RILEY,NaN


In [5]:
train_df = dump_df[dump_df['data_type'].str.endswith("Training")].drop('data_type', axis=1)
test_df = dump_df[dump_df['data_type'].str.endswith("Holdout")].drop('data_type', axis=1)
print(f"Train df: {train_df.shape}; Test df: {test_df.shape}")

Train df: (1527904, 246); Test df: (381976, 246)


In [6]:
train_df.to_csv("out/dump20241004_train.csv")
test_df.to_csv("out/dump20241004_test.csv")